#### Notebook to feature engineer fraction of time ozone was above 90th percentile

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [2]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2011','2011_multi_level_26_variables.nc')
ml_26_variables = xr.open_dataset(PATH_1) ## multilevel 26 variables
ml_26_variables = ml_26_variables.squeeze(dim="model_level") ## remove the model_level dimension
ml_26_variables=ml_26_variables.drop_vars('model_level') ## the dropped dimension becomes a column, so remove that
ml_26_variables['longitude'] = ml_26_variables['longitude']-360 ## necessary because of longitude data in multi-level case
ml_26_variables
ozone=ml_26_variables['go3'] ## get only ozone variable
ozone

<xarray.DataArray 'go3' (valid_time: 2920, latitude: 34, longitude: 78)> Size: 31MB
[7743840 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2011-01-01 ... 2011-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/31)
    GRIB_paramId:                             210203
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_pv:                                  [0.00000000e+00 2.00000000e+01 ...
    GRIB_shortName:                           go3
    GRIB_units:                               kg kg**-1
    long_name:                                Ozone mass mixing ratio (full c...
    units:                                    kg kg**-1
    standard_name:                            mass_fraction_of_ozone_in_air

In [3]:
ozone.to_dataframe()

go3
valid_time          latitude longitude                              
2011-01-01 00:00:00 49.25    -124.849999999999994  0.000000045346191
                             -124.099999999999994  0.000000041209560
                             -123.349999999999994  0.000000035562636
                             -122.599999999999994  0.000000032295475
                             -121.849999999999994  0.000000027435071
...                                                              ...
2011-12-31 21:00:00 24.50    -70.100000000000023   0.000000050719809
                             -69.350000000000023   0.000000052182269
                             -68.600000000000023   0.000000052467261
                             -67.850000000000023   0.000000052323049
                             -67.100000000000023   0.000000052322338

[7743840 rows x 1 columns]

In [4]:
PATH_2=os.path.join('..','CAMS_79_variables_2011.pkl')
get_data = pd.read_pickle(PATH_2) ## load the data with all variables
get_data.head()

,NAME,fips,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,tcso2,tcwv,co,aermr04,aermr05,aermr06,c2h6,hcho,h2o2,aermr09,aermr07,aermr10,aermr08,oh,c5h8,hno3,no2,no,go3,pan,c3h8,aermr01,aermr02,aermr03,q,aermr11,so2,t,lai_hv,lai_lv,asn,sd,tcw,aermssdus,aermssdum,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
1494,Autauga,1001,0.222903142185441,0.208854702718582,284.432999722863372,291.738658387209114,0.00941486490701,0.008129060270989,0.984864700853872,101728.100373019246035,0.089736782781202,0.000000010996987,0.000000013847045,0.000000019259719,0.00886249124226,0.076787852605938,1438.228761863982754,99987.326857107982505,0.235865475494415,0.192931146223771,0.147648233670896,0.101736627734521,0.058160089067154,0.000876186304338,0.000003277936924,0.000004396539779,0.000009445659017,0.000000004882269,0.000004010339129,0.009889691729096,0.000005510535506,0.000003161678006,0.000000453607919,0.006220520169684,0.000009252455558,0.000000620902083,0.000003131017519,25.046798539344927,0.000000178543848,0.000000000450964,0.000000000754089,0.000000000298524,0.000000000569874,0.000000004262341,0.000000000935125,0.000000000248853,0.000000007570371,0.000000000210155,0.000000003711973,0.000000000000022,0.0000000228099,0.0000000009242,0.000000007618401,0.000000000603382,0.000000059688558,0.000000003140233,0.000000000253157,0.000000000027469,0.000000002226016,0.000000000357245,0.009493987843734,0.000000002666455,0.000000006725148,291.60403835797581,3.880760790810764,2.624637573552741,0.87863205487649,0.007297349022118,25.139497250763817,0.000001532926342,0.000003030313,0.000003286726669,0.000000157102093,0.000002494599935,0.000000842318091,0.000016300907754,0.000000232893842,0.000018488158525,0.000001320272122,0.000008794422035,0.000001826115928
2005,Baldwin,1003,0.09291150018603,0.31945072782912,286.9092837088632,293.064779310994254,0.009503604186722,0.009508663078812,0.81519552148711,101715.059112138435012,0.087004549251509,0.000000010217312,0.000000013401248,0.000000018596263,0.011350049905456,0.074204435803247,454.970682925989649,101162.546667697897647,0.233290309666,0.191571342474217,0.147615606605046,0.103169503394227,0.060576137417536,0.000863634397455,0.000003249708997,0.000003885859429,0.000009842104255,0.000000004908674,0.000002763675421,0.010009143380804,0.000005548439732,0.000002834316561,0.000000420656716,0.006156037677418,0.000008476606524,0.000000581170376,0.000002446268927,26.80688384373466,0.000000165075969,0.000000000568846,0.000000000939555,0.000000000322425,0.000000000553571,0.000000003261566,0.000000001004626,0.000000000249279,0.000000006805509,0.000000000270327,0.000000003403587,0.000000000000028,0.000000015354148,0.000000001003609,0.000000006146517,0.000000000399122,0.000000061782627,0.000000002481288,0.000000000211493,0.000000000050757,0.000000004161007,0.000000001542248,0.010830943025345,0.000000002208196,0.00000000348171,292.986300864467808,3.888833666218919,2.646649475646851,0.879861039659569,0.005701287767721,26.88102308904341,0.000001824781502,0.000003604994553,0.000003497101203,0.000000156641259,0.000002220411989,0.000000852347656,0.000016080225159,0.000000288054602,0.000023070605018,0.000002269097505,0.00000870370171,0.000001125907913
127,Barbour,1005,0.282563984153067,0.100483435749544,284.532702188989049,292.425317915049334,0.009468574523291,0.008160808527996,0.987905779001801,101726.959626846073661,0.087682012857818,0.000000011039536,0.000000013886514,0.00000001926021,0.009362330031318,0.072247242827843,1256.402866916467246,100206.647901680407813,0.228373379039349,0.186921022906723,0.143265918714076,0.099075615469791,0.057087546689355,0.000871948236321,0.000003268408129,0.00000437763686,0.00000960458359,0.000000004873532,0.000003946721551,0.009912372563286,0.00000543462946,0.00000

In [5]:
percentile_75 = get_data['go3'].quantile(0.90) ## calculate 90th percentile
print(percentile_75)
threshold=percentile_75

5.9160671952701936e-08


In [6]:
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2011_county_shapefile','tl_2011_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,72,045,01804502,72045,Comerío,Comerío Municipio,13,H1,G4020,490,41980,None,A,73557132,319735,+18.2250402,-066.2202984,"POLYGON ((-66.25713 18.24744, -66.25695 18.247..."
1,72,009,01804484,72009,Aibonito,Aibonito Municipio,13,H1,G4020,490,41980,None,A,81094444,71897,+18.1306472,-066.2639743,"POLYGON ((-66.25753 18.07588, -66.25937 18.077..."
2,72,039,01804499,72039,Ciales,Ciales Municipio,13,H1,G4020,490,41980,None,A,172309964,459615,+18.2912286,-066.5130169,"POLYGON ((-66.55771 18.27506, -66.55782 18.275..."
3,72,003,01804481,72003,Aguada,Aguada Municipio,13,H1,G4020,None,10380,None,A,79904253,38043835,+18.3756809,-067.1837074,"POLYGON ((-67.16355 18.41748, -67.16190 18.416..."
4,72,151,01804556,72151,Yabucoa,Yabucoa Municipio,13,H1,G4020,490,41980,None,A,143005203,72593008,+18.0598615,-065.8598555,"POLYGON ((-65.87885 18.02066, -65.87926 18.020..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,42,099,01213682,42099,Perry,Perry County,06,H1,G4020,276,25420,None,A,1428280078,10597982,+40.3977795,-077.2663284,"POLYGON ((-77.02920 40.38688, -77.02918 40.386..."
3230,40,061,01101818,40061,Haskell,Haskell County,06,H1,G4020,None,None,None,A,1493228577,126266671,+35.2322937,-095.1095776,"POLYGON ((-95.22136 35.27282, -95.22103 35.273..."
3231,17,197,01785190,17197,Will,Will County,06,H1,G4020,176,16980,16974,A,2167580642,31904479,+41.4484742,-087.9784564,"POLYGON ((-88.25535 41.55009, -88.25539 41.550..."
3232,40,081,01101828,40081,Lincoln,Lincoln County,06,H1,G4020,416,36420,None,A,2466481584,34614223,+35.7031175,-096.8813917,"POLYGON ((-97.14113 35.60848, -97.14119 35.609..."


In [7]:
geometry_column=county_gdf['geometry']
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x                   y
0    0    -66.257125000000002  18.247439000000000
     1    -66.256954999999991  18.247450999999998
     2    -66.256174999999999  18.247370999999998
     3    -66.255698999999993  18.247496999999999
     4    -66.255297999999996  18.247540999999998
...                       ...                 ...
3233 1161 -81.391229999999993  41.463234000000000
     1162 -81.391228999999996  41.463273000000001
     1163 -81.391212999999993  41.463906000000001
     1164 -81.391134999999991  41.466968000000001
     1165 -81.391133999999994  41.467050999999998

[7394987 rows x 2 columns]

In [8]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time above percentile
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= ozone.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['go3'].values[i] > threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'ozone above percentile':[frac_time]})


    return pm_above_df

In [9]:
len(county_gdf)

3234

In [10]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 1h 52min 50s
Wall time: 1h 54min 26s


In [11]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

,ozone above percentile
0,0.000000000000000
1,0.000000000000000
2,0.000000000000000
3,0.000000000000000
4,0.000000000000000
...,...
3229,38.904109589041099
3230,45.753424657534246
3231,24.280821917808218
3232,49.691780821917810


In [12]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','GEOID','NAMELSAD','LSAD','CLASSFP','MTFCC'
                             ,'CSAFP','CBSAFP','METDIVFP','FUNCSTAT','ALAND','AWATER','INTPTLAT',
                             'INTPTLON'], axis=1)
county_gdf

,NAME,geometry,fips
0,Comerío,"POLYGON ((-66.25713 18.24744, -66.25695 18.247...",72045
1,Aibonito,"POLYGON ((-66.25753 18.07588, -66.25937 18.077...",72009
2,Ciales,"POLYGON ((-66.55771 18.27506, -66.55782 18.275...",72039
3,Aguada,"POLYGON ((-67.16355 18.41748, -67.16190 18.416...",72003
4,Yabucoa,"POLYGON ((-65.87885 18.02066, -65.87926 18.020...",72151
...,...,...,...
3229,Perry,"POLYGON ((-77.02920 40.38688, -77.02918 40.386...",42099
3230,Haskell,"POLYGON ((-95.22136 35.27282, -95.22103 35.273...",40061
3231,Will,"POLYGON ((-88.25535 41.55009, -88.25539 41.550...",17197
3232,Lincoln,"POLYGON ((-97.14113 35.60848, -97.14119 35.609...",40081


In [13]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,ozone above percentile
0,Comerío,"POLYGON ((-66.25713 18.24744, -66.25695 18.247...",72045,0.000000000000000
1,Aibonito,"POLYGON ((-66.25753 18.07588, -66.25937 18.077...",72009,0.000000000000000
2,Ciales,"POLYGON ((-66.55771 18.27506, -66.55782 18.275...",72039,0.000000000000000
3,Aguada,"POLYGON ((-67.16355 18.41748, -67.16190 18.416...",72003,0.000000000000000
4,Yabucoa,"POLYGON ((-65.87885 18.02066, -65.87926 18.020...",72151,0.000000000000000
...,...,...,...,...
3229,Perry,"POLYGON ((-77.02920 40.38688, -77.02918 40.386...",42099,38.904109589041099
3230,Haskell,"POLYGON ((-95.22136 35.27282, -95.22103 35.273...",40061,45.753424657534246
3231,Will,"POLYGON ((-88.25535 41.55009, -88.25539 41.550...",17197,24.280821917808218
3232,Lincoln,"POLYGON ((-97.14113 35.60848, -97.14119 35.609...",40081,49.691780821917810


In [14]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,ozone above percentile
0,Comerío,"POLYGON ((-66.25713 18.24744, -66.25695 18.247...",72045,0.0
1,Aibonito,"POLYGON ((-66.25753 18.07588, -66.25937 18.077...",72009,0.0
2,Ciales,"POLYGON ((-66.55771 18.27506, -66.55782 18.275...",72039,0.0
3,Aguada,"POLYGON ((-67.16355 18.41748, -67.16190 18.416...",72003,0.0
4,Yabucoa,"POLYGON ((-65.87885 18.02066, -65.87926 18.020...",72151,0.0


In [15]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,ozone above percentile
0,Comerío,72045,0.000000000000000
1,Aibonito,72009,0.000000000000000
2,Ciales,72039,0.000000000000000
3,Aguada,72003,0.000000000000000
4,Yabucoa,72151,0.000000000000000
...,...,...,...
3229,Perry,42099,38.904109589041099
3230,Haskell,40061,45.753424657534246
3231,Will,17197,24.280821917808218
3232,Lincoln,40081,49.691780821917810


In [16]:
county_var.to_pickle('ozone_above_percentile.pkl')